In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle

In [2]:
data_generation_process = "SPO_Data_Generation"
# data_generation_process = "DDR_Generation"

In [3]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/SPO_Data_Generation/


# Parameters

In [4]:
# import pyepo
# generate data
grid = (5,5) # grid size
num_data = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.5 # polynomial degree
e = 1.0 # noise width

In [5]:
DataPath = DataPath + "data_size="+str(num_data)+"_deg="+str(deg)+"_e="+str(e)+"/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

In [6]:
def obtain_data(data_generation_process,num_data,num_test, num_feat, grid, deg, e, seed):
    from Data import data_generation
    data_gen = data_generation()
    if data_generation_process == "SPO_Data_Generation":
        feats, costs = data_gen.generate_Shortest_Path_Data(num_data+num_test, num_feat, grid, deg, e, seed=seed)
        # split train test data
        from sklearn.model_selection import train_test_split
        x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test, random_state=42)

    if data_generation_process == "DDR_Generation":
        lower = 0
        upper = 1
        p = 5
        d = 40
        alpha = 1.0
        mis = deg
        n_epsilon = 1
        W_star = data_gen.generate_truth("",lower, upper, p, d, seed,version = 0) 
        # print("W_star = ",W_star[0,:])
        x_test, z_test_ori, c_test, x_train, z_train_ori, c_train, W_star = data_gen.generate_samples("",p, d, num_test, num_data, alpha, W_star, n_epsilon, mis, thres = 10, 
                                version = 1, x_dist = 'normal', e_dist = 'normal', x_low = 0, x_up = 2, x_mean = 2, x_var = 0.25, bump = 0) 

    return x_train, x_test, c_train, c_test

In [ ]:
seed_all = np.arange(1,10)
cost_Oracle_all = {}; cost_SPO_all = {}; cost_OLS_all = {}; cost_DDR_all = {}

for seed in seed_all:
    DataPath_seed = DataPath +"Seed="+str(seed)+"/"
    pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)

    # #  ****** Data generation *********
    x_train, x_test, c_train, c_test = obtain_data(data_generation_process,num_data,num_test, num_feat, grid, deg, e, seed)

    raw_data = {}
    raw_data["x_train"] = x_train; raw_data["x_test"] = x_test; raw_data["c_train"] = c_train; raw_data["c_test"] = c_test
    with open(DataPath_seed +'raw_data.pkl', "wb") as tf:
        pickle.dump(raw_data,tf)

    #  ****** SPO *********
    print("*** seed = ",seed,": Run SPO ========")
    from SPO_Plus import run_SPO_Shortest_Path
    SPO_runner = run_SPO_Shortest_Path()
    batch_size = 1
    num_epochs = 1
    arcs,loader_train,loader_test,cost_Oracle_all[seed],cost_SPO_all[seed] = SPO_runner.run(DataPath_seed,x_train,c_train,x_test,c_test,batch_size,num_feat,grid,num_epochs,True)
    print("Average Oracle Cost = ",np.mean(cost_Oracle_all[seed]))
    print("Average SPO Cost = ",np.mean(cost_SPO_all[seed]))
    print()

    #  ****** OLS *********
    print("*** seed = ",seed,": Run OLS ========")
    from OLS import run_OLS_Shortest_Path
    OLS_runner = run_OLS_Shortest_Path()
    cost_OLS_all[seed] = OLS_runner.run(DataPath_seed,arcs,x_train,c_train,grid,loader_test,loader_train)
    print("Average OLS Cost = ",np.mean(cost_OLS_all[seed]))
    print()

    #  ****** DDR *********
    print("*** seed = ",seed,": Run DDR ========")
    from DDR import run_DDR_Shortest_Path
    DDR_runner = run_DDR_Shortest_Path()
    mu_arr = np.arange(0.0,0.51,0.05)
    lamb_arr = np.arange(1,4.1,0.5)
    # mu_arr = [0.5]
    # lamb_arr = [0.1,1.0,10]
    cost_DDR_all[seed] = DDR_runner.run(DataPath_seed,lamb_arr,mu_arr,arcs,x_train, c_train, grid,loader_test,num_nodes=25)
    print()

*** seed =  1 : Run SPO ========
Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2025-03-25
Test
Optimizing for optDataset...


100%|██████████| 100/100 [00:00<00:00, 1708.77it/s]


Test
Optimizing for optDataset...


100%|██████████| 1000/1000 [00:00<00:00, 2185.15it/s]


Num of cores: 2
Average Oracle Cost =  4.658342152833939
Average SPO Cost =  7.080734337605215

*** seed =  1 : Run OLS ========
Average OLS Cost =  6.927368302423703

*** seed =  1 : Run DDR ========
lambda =  1.0 , mu =  0.0 , Average DDR cost =  6.948242479914181

lambda =  1.0 , mu =  0.05 , Average DDR cost =  6.95392526934691

lambda =  1.0 , mu =  0.1 , Average DDR cost =  6.956431036455099

lambda =  1.0 , mu =  0.15000000000000002 , Average DDR cost =  6.9107167106696945

lambda =  1.0 , mu =  0.2 , Average DDR cost =  6.906816532835837

lambda =  1.0 , mu =  0.25 , Average DDR cost =  6.900678914333235

lambda =  1.0 , mu =  0.30000000000000004 , Average DDR cost =  6.877292861475617

lambda =  1.0 , mu =  0.35000000000000003 , Average DDR cost =  6.894597584966345

lambda =  1.0 , mu =  0.4 , Average DDR cost =  6.879524982813753

lambda =  1.0 , mu =  0.45 , Average DDR cost =  6.899162887211896

lambda =  1.0 , mu =  0.5 , Average DDR cost =  6.891072685538217

lambda =  1

...

Num of cores: 2
Average Oracle Cost =  4.7275598778724675
Average SPO Cost =  7.146225646285748

*** seed =  5 : Run OLS ========
Average OLS Cost =  6.97560201691481

*** seed =  5 : Run DDR ========
lambda =  1.0 , mu =  0.0 , Average DDR cost =  7.158044443351275

lambda =  1.0 , mu =  0.05 , Average DDR cost =  7.13196452060825

lambda =  1.0 , mu =  0.1 , Average DDR cost =  7.143901545050729

lambda =  1.0 , mu =  0.15000000000000002 , Average DDR cost =  7.108893817371864

lambda =  1.0 , mu =  0.2 , Average DDR cost =  7.093131474409892

lambda =  1.0 , mu =  0.25 , Average DDR cost =  7.068724381672859

lambda =  1.0 , mu =  0.30000000000000004 , Average DDR cost =  7.073839374388277

lambda =  1.0 , mu =  0.35000000000000003 , Average DDR cost =  7.041937547985174

lambda =  1.0 , mu =  0.4 , Average DDR cost =  7.023874642640934

lambda =  1.0 , mu =  0.45 , Average DDR cost =  7.004882731745136

lambda =  1.0 , mu =  0.5 , Average DDR cost =  7.001248354409996

lambda =  1.

100%|██████████| 100/100 [00:00<00:00, 1383.62it/s]


Test
Optimizing for optDataset...


100%|██████████| 1000/1000 [00:00<00:00, 1960.67it/s]


Num of cores: 2
Average Oracle Cost =  4.6718733323812485
Average SPO Cost =  7.154671416052908

*** seed =  6 : Run OLS ========
Average OLS Cost =  6.958839252466183

*** seed =  6 : Run DDR ========
lambda =  1.0 , mu =  0.0 , Average DDR cost =  7.0252581600034265

lambda =  1.0 , mu =  0.05 , Average DDR cost =  7.018601312009276

lambda =  1.0 , mu =  0.1 , Average DDR cost =  7.012046851384936

lambda =  1.0 , mu =  0.15000000000000002 , Average DDR cost =  6.968276532062962



# Results

In [ ]:
for seed in seed_all:
    cost_ddr_opt = 10000
    lamb_opt = 0
    mu_opt = 0
    ddr_ols_ratio = 0
    for lamb in lamb_arr:
        for mu in mu_arr:
            # print("Seed = ",seed,", lamb_opt = ",lamb,",mu_opt = ",mu,", Average DRR Cost = ", np.round(np.mean(cost_DDR_all[seed][lamb,mu]["cost"]),4))
            if np.mean(cost_DDR_all[seed][lamb,mu]["cost"]) < cost_ddr_opt:
                cost_ddr_opt = np.mean(cost_DDR_all[seed][lamb,mu]["cost"])
                lamb_opt = lamb
                mu_opt = mu
                ddr_ols_ratio = ((np.mean(cost_OLS_all[seed])) - cost_ddr_opt)/(np.mean(cost_OLS_all[seed]) - np.mean(cost_Oracle_all[seed]))
    print("Seed = ",seed,", lamb_opt = ",lamb_opt,",mu_opt = ",mu_opt,", Average DRR Cost = ", np.round(cost_ddr_opt,4), "DDR_OLS_ratio = ",ddr_ols_ratio)
